In [1]:
pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 20.3 MB/s eta 0:00:00


In [2]:
pdf_path = "/content/drive/MyDrive/빅프/2019_영어_홀수형.pdf"

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

text = extract_text_from_pdf(pdf_path)
print(text[:1000])  # 텍스트의 처음 1000자를 출력하여 확인

1
8
  1번부터 17번까지는 듣고 답하는 문제입니다. 1번부터 
15번까지는 한 번만 들려주고, 16번부터 17번까지는 두 번 
들려줍니다. 방송을 잘 듣고 답을 하시기 바랍니다.
1. 대화를 듣고, 남자의 마지막 말에 대한 여자의 응답으로 가장 
적절한 것을 고르시오.
①No. You can’t study with us.
②Okay. I’ll do the report by myself. 
③Sure. I’ll call you when I’m done.
④Yes. I’m pleased to join your team.
⑤Sorry. You have to finish by tomorrow.
2. 대화를 듣고, 여자의 마지막 말에 대한 남자의 응답으로 가장 
적절한 것을 고르시오.
①Be careful. The roads are slippery.
②I agree. The seats are very comfortable.
③Wonderful. Let’s attend the program together.
④Great. I’ll register my son for the program. 
⑤I’m sorry. Your son has to wait longer.
3. 다음을 듣고, 남자가 하는 말의 목적으로 가장 적절한 것을 
고르시오.
①경기 취소를 공지하려고
②팬클럽 가입을 권유하려고
③경기장 개장을 홍보하려고
④웹 사이트 점검을 안내하려고
⑤시상식 일정 변경을 사과하려고
4. 대화를 듣고, 여자의 의견으로 가장 적절한 것을 고르시오.
①실패한 실험을 분석하면 실험에 성공할 수 있다.
②과학 수업에서는 이론과 실습이 병행되어야 한다.
③과학자가 되기 위해서는 인문학적 소양도 필요하다.
④실험 일지는 실험 보고서 작성에 도움이 된다.
⑤실험을 할 때마다 안전 교육을 해야 한다.
5. 대화를 듣고, 두 사람의 관계를 가장 잘 나타낸 것을 고르시오.
①모델－사진작가
②기증자－박물관 직원
③영화 관람객－티켓 판매원
④인테리어 디자이너－건축가
⑤고객－가구점 직원 
6. 

In [4]:
import re

def split_text_into_components(text):
    questions = []
    choices = []
    passages = []

    current_question = None
    current_choices = []
    current_passage = ""

    question_pattern = re.compile(r'^\d+\.\s')
    choice_pattern = re.compile(r'^[①②③④⑤]')
    unwanted_text_pattern = re.compile(r'(이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다.|홀수형|짝수형|\d+\s?페이지)')
    passage_start_pattern = re.compile(r'[A-Za-z]')

    # 불필요한 텍스트 제거
    text = unwanted_text_pattern.sub('', text)

    lines = text.split('\n')
    skip_until_question_18 = True
    assembling_question = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if skip_until_question_18:
            if line.startswith("18."):
                skip_until_question_18 = False
            else:
                continue

        if question_pattern.match(line):
            if '도표' in line:
                current_question = None
                current_choices = []
                current_passage = ""
                assembling_question = False
                continue

            if current_question or current_choices:
                questions.append(current_question.strip())
                choices.append(current_choices)
                passages.append(current_passage.strip() if current_passage else "")
                current_passage = ""
            current_question = line
            current_choices = []
            assembling_question = True
        elif choice_pattern.match(line):
            assembling_question = False
            current_choices.append(line)
        elif passage_start_pattern.match(line) and not current_choices:
            current_passage += line + ' '
        else:
            if current_choices:
                current_choices[-1] += ' ' + line  # 선지 내용이 여러 줄일 경우 이어 붙임
            elif assembling_question:
                current_question += ' ' + line  # 질문이 여러 줄일 경우 이어 붙임

    if current_question or current_choices:
        questions.append(current_question.strip())
        choices.append(current_choices)
        passages.append(current_passage.strip() if current_passage else "")

    return passages, questions, choices

In [5]:
def process_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if text:
        passages, questions, choices = split_text_into_components(text)
        return passages, questions, choices
    else:
        return [], [], []

In [9]:
passages, questions, choices = process_pdf(pdf_path)

print(f"Question: {questions[1]}")
print(f"Passage: {passages[1]}")
print("Options:")
for choice in choices[1]:
    print(choice)
print()

Question: 19. 다음 글에 드러난 Dave의 심경 변화로 가장 적절한 것은?
Passage: The waves were perfect for surfing. Dave, however, just could not stay on his board. He had tried more than ten times to stand up but never managed it. He felt that he would never succeed. He was about to give up when he looked at the sea one last time. The swelling waves seemed to say, “Come on, Dave. One more try!” Taking a deep breath, he picked up his board and ran into the water. He waited for the right wave. Finally, it came. He jumped up onto the board just like he had practiced. And this time, standing upright, he battled the wave all the way back to shore. Walking out of the water joyfully, he cheered, “Wow, I did it!”
Options:
①frustrated →delighted
②bored →comforted
③calm →annoyed
④relieved →frightened
⑤pleased →upset



In [ ]:
import pandas as pd

def save_to_csv(passages, questions, choices, output_file):
    data = {
        "질문": questions,
        "선지": ['\n'.join(choice) for choice in choices],
        "지문": passages,
    }
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

In [ ]:
def make_pdf(pdf_path, output_file):
    text = extract_text_from_pdf(pdf_path)
    passages, questions, choices = split_text_into_components(text)
    save_to_csv(passages, questions, choices, output_file)

In [ ]:
output_file = "2019.csv"
make_pdf(pdf_path, output_file)

### 답안 추출

In [10]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):  # 전체 페이지 반복
        page = document.load_page(page_num)
        text += page.get_text() + "\n"
    return text

In [11]:
pdf_path = '/content/drive/MyDrive/빅프/영어 데이터/2019_영어_정답지.pdf'

In [12]:
# PDF에서 전체 텍스트 추출
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text[:1000])  # 텍스트의 처음 1000자를 출력하여 확인

2019학년도 대학수학능력시험
영어
영어
영어영역
영역
영역 
 
 정답 및 해설
정답 및 해설
정답 및 해설
1
01. ③  02. ④　 03. ①  04. ①  05. ②  06. ④　 07.　⑤  08. ②  09. ②  10. ④ 
11. ⑤  12. ④  13. ④  14. ⑤  15. ①  16. ③  17. ⑤  18. ③  19. ①  20. ①
21. ①  22. ④  23. ④  24. ①  25. ③  26. ④  27. ⑤  28. ④  29. ②  30. ⑤
31. ③  32. ①  33. ②  34. ②  35. ④  36. ③  37. ⑤  38. ③  39. ③  40. ①
41. ③  42. ②  43. ②  44. ⑤  45. ②  
1. [출제 의도] 짧은 응답 고르기
 [해석]
남: Amy, 너 오늘 밤 Donna의 집에서 공부한다고 했지, 그렇지?
여: 네, 아빠. 자정까지 저희 팀 보고서를 온라인으로 제출해야 해요.
남: 네가 꽤 늦을 것 같구나. 내가 널 데리러 갈까?
여: 좋아요. 제가 끝나면 전화 드릴게요.
[풀이]
팀원의 집에서 늦게까지 보고서를 작성해서 자정까지 제출해야 하는 Amy에게 아빠
가 데리러 가겠다고 말했을 때, 딸 Amy의 응답으로 가장 적절한 것은 ③ ‘좋아요. 
제가 끝나면 전화 드릴게요.’이다.
① 아니요. 아빠는 저희와 함께 공부할 수 없어요.
② 알았어요. 제가 보고서를 혼자서 해 볼게요.
④ 네. 저는 아빠 팀에 들어와서 기뻐요.
⑤ 미안해요. 아빠는 내일까지 마쳐야 해요.
[Words and Phrases]
submit 제출하다  by midnight 자정까지
 
2. [출제 의도] 짧은 응답 고르기
 [해석]
여: 기다려 주셔서 감사합니다, 손님. 무엇을 도와드릴까요?
남: 제 아들이 도로 안전 프로그램에 참여하고 싶어 해요. 아직 남아있는 자리가 있  
    나요?
여: 운 좋은 날이군요! 누군가가 방금 취소했어요. 그러니 아드님이 그 자리에 들어올 
수 있어요.
남: 잘됐네요. 제

답만 추출

In [13]:
import fitz

In [16]:
def extract_answers(text):
    answers = []

    answer_pattern = re.compile(r'\d{2}\.\s([①②③④⑤])')

    # 텍스트를 줄 단위로 분리
    lines = text.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue
        answer_matches = answer_pattern.findall(line)
        if answer_matches:
            # 정답을 감지
            answers.extend(answer_matches)

    return answers

def print_answers(answers):
    for answer in answers:
        print(answer)

In [17]:
# 정답만 추출
answers = extract_answers(pdf_text)

# 정답 출력
print_answers(answers)

③
④
①
①
②
④
⑤
②
②
④
⑤
④
④
⑤
①
③
⑤
③
①
①
①
④
④
①
③
④
⑤
④
②
⑤
③
①
②
②
④
③
⑤
③
③
①
③
②
②
⑤
②


In [18]:
def extract_explanations(text):
    explanations = []
    current_explanation = None
    in_explanation = False

    # 패턴 정의: '[해석]' 또는 '[풀이]'로 시작하는 텍스트
    explanation_start_pattern = re.compile(r'^\[(해석|풀이)\]')
    explanation_end_pattern = re.compile(r'^\[\w+\]|\d{2}\.\s[①②③④⑤]')
    unwanted_text_pattern = re.compile(r'(이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다.|홀수형|짝수형|\d+\s?페이지)')

    # 불필요한 텍스트 제거
    text = unwanted_text_pattern.sub('', text)

    # 텍스트를 줄 단위로 분리
    lines = text.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if explanation_start_pattern.match(line):
            if current_explanation:
                explanations.append(current_explanation.strip())
            current_explanation = line
            in_explanation = True
        elif explanation_end_pattern.match(line):
            if current_explanation:
                explanations.append(current_explanation.strip())
            current_explanation = None
            in_explanation = False
        elif in_explanation:
            current_explanation += " " + line

    if current_explanation:
        explanations.append(current_explanation.strip())

    return explanations

def print_explanations(explanations):
    for explanation in explanations:
        print(explanation)
        print('-' * 50)

In [19]:
# 설명만 추출
explanations = extract_explanations(pdf_text)

# 설명 출력
print_explanations(explanations)

[해석] 남: Amy, 너 오늘 밤 Donna의 집에서 공부한다고 했지, 그렇지? 여: 네, 아빠. 자정까지 저희 팀 보고서를 온라인으로 제출해야 해요. 남: 네가 꽤 늦을 것 같구나. 내가 널 데리러 갈까? 여: 좋아요. 제가 끝나면 전화 드릴게요.
--------------------------------------------------
[풀이] 팀원의 집에서 늦게까지 보고서를 작성해서 자정까지 제출해야 하는 Amy에게 아빠 가 데리러 가겠다고 말했을 때, 딸 Amy의 응답으로 가장 적절한 것은 ③ ‘좋아요. 제가 끝나면 전화 드릴게요.’이다. ① 아니요. 아빠는 저희와 함께 공부할 수 없어요. ② 알았어요. 제가 보고서를 혼자서 해 볼게요. ④ 네. 저는 아빠 팀에 들어와서 기뻐요. ⑤ 미안해요. 아빠는 내일까지 마쳐야 해요. [Words and Phrases] submit 제출하다  by midnight 자정까지 2. [출제 의도] 짧은 응답 고르기
--------------------------------------------------
[해석] 여: 기다려 주셔서 감사합니다, 손님. 무엇을 도와드릴까요? 남: 제 아들이 도로 안전 프로그램에 참여하고 싶어 해요. 아직 남아있는 자리가 있 나요? 여: 운 좋은 날이군요! 누군가가 방금 취소했어요. 그러니 아드님이 그 자리에 들어올 수 있어요. 남: 잘됐네요. 제 아들을 그 프로그램에 등록할게요.
--------------------------------------------------
[풀이] 아들을 도로 안전 프로그램에 참여시키고 싶은 남자가 아직 자리가 있는지 묻자 여자 는 누군가가 취소해서 자리가 있다고 말했을 때 남자의 응답으로 가장 적절한 것은 ④ ‘잘됐네요. 제 아들을 그 프로그램에 등록할게요.’이다. 2 ① 조심하세요. 길이 미끄러워요. ② 동의해요. 그 좌석들은 매우 편안해요. ③ 대단하군요. 함께 그 프로그램에 참여하지요. ⑤ 죄송해요. 아드님은 더 오래 기다려야 해요. [Words a